## This experiment was performed using LightFM which a very popular recommender module and it has support to take in different data modalities such as text, image, graphical, etc. Please check this out their official documentation in the link mentioned below:  

Link to LightFM:
making.lyst.com/lightfm/docs/home.html

Hope you like this notebook, please feel free to vote for this notebook

## Importing Required Libraries

In [55]:
# Importing Libraries
import sys, os
import re
import tqdm
import time
import pickle
import random
import itertools

import pandas as pd
import numpy as np
import scipy.sparse as sparse
%matplotlib inline
import matplotlib.pyplot as plt

# lightfm 
from lightfm import LightFM # model
from lightfm.evaluation import precision_at_k
from lightfm.cross_validation import random_train_test_split

# multiprocessing for inferencing
from multiprocessing import Pool

In [2]:
os.environ["openblas_set_num_threads"] = "1"
data_path = r'../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv'
submission_data_path = r'../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv'

In [3]:
# Data Extraction
def create_data(datapath):
    df=pd.read_csv(datapath)
    return df

In [4]:
# Loal all sales data (for 3 years startign from 2018 to 2020)
transactions_data=create_data(data_path)
print(transactions_data.shape)

# # Unique Attributes
print(str(len(transactions_data['t_dat'].drop_duplicates())) + "-total No of unique transactions dates in data sheet")
print(str(len(transactions_data['customer_id'].drop_duplicates())) + "-total No of unique customers ids in data sheet")
print(str(len(transactions_data['article_id'].drop_duplicates())) + "-total No of unique article ids courses names in data sheet")
print(str(len(transactions_data['sales_channel_id'].drop_duplicates())) + "-total No of unique sales channels in data sheet")

(31788324, 5)
734-total No of unique transactions dates in data sheet
1362281-total No of unique customers ids in data sheet
104547-total No of unique article ids courses names in data sheet
2-total No of unique sales channels in data sheet


In [5]:
transactions_data.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [6]:
transactions_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   t_dat             object 
 1   customer_id       object 
 2   article_id        int64  
 3   price             float64
 4   sales_channel_id  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 1.2+ GB


## Aggregating Customers and Articles irrespective of transaction dates

In [7]:
transactions_data = transactions_data.groupby(['customer_id','article_id']).agg({'price':'sum','t_dat':'count'}).reset_index()
transactions_data = transactions_data[['customer_id','article_id','price']]

## Generating user and article index mapping dictionaries

In [8]:
def get_customers_list():
    # Creating a list of users
    return np.sort(transactions_data['customer_id'].unique())

def get_articles_list():
    # Creating a list of courses 
    item_list = transactions_data['article_id'].unique()
    return item_list

def id_mappings(customers_list, articles_list):
    """
    
    Create id mappings to convert user_id, item_id, and feature_id
    
    """
    customer_to_index_mapping = {}
    index_to_customer_mapping = {}
    for customer_index, customer_id in enumerate(customers_list):
        customer_to_index_mapping[customer_id] = customer_index
        index_to_customer_mapping[customer_index] = customer_id
        
    article_to_index_mapping = {}
    index_to_article_mapping = {}
    for article_index, article_id in enumerate(articles_list):
        article_to_index_mapping[article_id] = article_index
        index_to_article_mapping[article_index] = article_id
        
    return customer_to_index_mapping, index_to_customer_mapping, \
           article_to_index_mapping, index_to_article_mapping

In [9]:
customers = get_customers_list()
articles = get_articles_list()

In [10]:
customers

array(['00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657',
       '0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa',
       '000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318',
       ...,
       'ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1778d0116cffd259264',
       'ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38b2236865d949d4df6a',
       'ffffd9ac14e89946416d80e791d064701994755c3ab686a1eaf3458c36f52241'],
      dtype=object)

In [11]:
articles

array([176209023, 568601006, 568601043, ..., 618853001, 533932004,
       568499001])

In [12]:
# Generate mapping, LightFM library can't read other than (integer) index
customer_to_index_mapping, index_to_customer_mapping, \
article_to_index_mapping, index_to_article_mapping = id_mappings(customers, articles)

## Generate Customer Article Interaction Matrix

In [13]:
def get_customer_article_interaction(customer_article_amt_df):
    #start indexing
    customer_article_amt_df["customer_id"] = customer_article_amt_df["customer_id"]
    customer_article_amt_df["article_id"] = customer_article_amt_df["article_id"]
    customer_article_amt_df["price"] = customer_article_amt_df["price"]

    # Preprocessing dataframe created
    customer_article_amt_df = customer_article_amt_df.rename(columns = {"price":"total_amount_spent"})

    # Replace Amount Column with category codes 
    customer_article_amt_df['total_amount_spent'] = customer_article_amt_df['total_amount_spent'].astype('category')
    customer_article_amt_df['total_amount_spent'] = customer_article_amt_df['total_amount_spent'].cat.codes

    return customer_article_amt_df

def get_interaction_matrix(df, df_column_as_row, df_column_as_col, 
                        df_column_as_value, row_indexing_map, col_indexing_map):
    
    row = df[df_column_as_row].apply(lambda x: row_indexing_map[x]).values
    col = df[df_column_as_col].apply(lambda x: col_indexing_map[x]).values
    value = df[df_column_as_value].values
    
    return sparse.coo_matrix((value, (row, col)), shape = (len(row_indexing_map), len(col_indexing_map)))


In [14]:
# Create customer and article interaction dataframe
customer_to_article = get_customer_article_interaction(customer_article_amt_df = transactions_data[['customer_id','article_id','price']])
print(customer_to_article.shape)                                                  

(27306439, 3)


In [15]:
customer_to_article.head()

,customer_id,article_id,total_amount_spent
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,176209023,8137
1,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601006,27464
2,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,12672
3,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,607642008,2052
4,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,625548001,10525


In [16]:
# Generate customer_article_interaction_matrix for train data
customer_to_article_interaction = get_interaction_matrix(customer_to_article, "customer_id", "article_id", "total_amount_spent", \
                                                    customer_to_index_mapping, article_to_index_mapping)

In [17]:
customer_to_article_interaction

<1362281x104547 sparse matrix of type '<class 'numpy.int32'>'
	with 27306439 stored elements in COOrdinate format>

## Light FM Model Training

In [18]:
#### FULL MODEL TRAINING ####
# Retraining the final model with full dataset
"""
Training model without any article or customer features
"""
final_model_without_feature = LightFM(loss = "warp")

# Fitting to combined dataset with pure collaborative filtering result
start = time.time() 
final_model_without_feature.fit(customer_to_article_interaction,
                                user_features=None, 
                                item_features=None, 
                                sample_weight=None, 
                                epochs=1, 
                                num_threads=4, 
                                verbose=False)
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

time taken = 19.72 seconds


## Recommendation sampling and Comparison with Known Positives

In [19]:
class recommendation_sampling():
    def __init__(self, model, items = None, user_to_product_interaction_matrix = None, 
                item_features = None, user2index_map = None):
        
        self.user_to_product_interaction_matrix = user_to_product_interaction_matrix
        self.item_features = item_features if item_features is not None else None
        self.model = model
        self.items = items
        self.user2index_map = user2index_map
    
    def recommendation_for_user(self, user, k=3, prediction_type = 'normal'):
        # Getting the userindex
        userindex = self.user2index_map.get(user, None)
        if userindex == None:
            print("User %s not provided during Training the model" %(user))
            return None

        # Products already bought
        known_positives = self.items[self.user_to_product_interaction_matrix.tocsr()[userindex].indices]
        
        # Scores from model prediction
        scores = self.model.predict(user_ids = userindex, item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1])) if prediction_type == 'normal' else \
            self.model.predict(user_ids = userindex, item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1]), item_features = self.item_features)
    
        # Top items
        top_items = self.items[np.argsort(-scores)]
        
        # Printing out the result
        print("User %s" % user)
        print("     Known positives:")
        for x in known_positives[:k]:
            print("                  %s" % x)
            
        print("     Recommended:")
        for x in top_items[:k]:
            print("                  %s" % x)

    def get_recommendation(self, user, k=3, prediction_type = 'normal'):
        # Getting the userindex
        userindex = self.user2index_map.get(user, None)
        if userindex == None:
            return None
        
        # Products already bought
        known_positives = self.items[self.user_to_product_interaction_matrix.tocsr()[userindex].indices]
        
        # Scores from model prediction
        scores = self.model.predict(user_ids = userindex, item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1])) if prediction_type == 'normal' else \
            self.model.predict(user_ids = userindex, item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1]), item_features = self.item_features)
        
        # Top items
        top_items = self.items[np.argsort(-scores)]

        # Returning results
        recommended_list, recommender_count = [],1
        for item in top_items[:k]:
            recommended_list.append({'Priority': recommender_count,'Article': item})
            recommender_count+=1
        return known_positives, recommended_list
    
    def get_batched_recommendation(self, user, k=3, prediction_type='normal'):
        # Getting user_indexes 
        user_index = self.user2index_map.get(user, None)
        if user_index is None:
            return None
        
        # Scores from model
        scores = self.model.predict(user_ids = user_index, item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1])) if prediction_type == 'normal' else \
            self.model.predict(user_ids = user_index, item_ids = np.arange(self.user_to_product_interaction_matrix.shape[1]), item_features = self.item_features)
    
        # Top items
        top_items = self.items[np.argsort(-scores)]
        
        return top_items[:k]

In [20]:
# Giving recommendations
recom_without_feature = recommendation_sampling(model = final_model_without_feature,
                                               items = articles,
                                               user_to_product_interaction_matrix = customer_to_article_interaction,
                                               user2index_map = customer_to_index_mapping)

In [21]:
recom_without_feature.recommendation_for_user('00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657')
recom_without_feature.recommendation_for_user('0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa')
recom_without_feature.recommendation_for_user('000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318')
recom_without_feature.recommendation_for_user('00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e')
recom_without_feature.recommendation_for_user('00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a')

User 00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657
     Known positives:
                  176209023
                  568601006
                  568601043
     Recommended:
                  372860001
                  706016001
                  372860002
User 0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa
     Known positives:
                  351484002
                  377277001
                  478646001
     Recommended:
                  706016001
                  706016002
                  759871002
User 000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318
     Known positives:
                  351484002
                  723529001
                  541518023
     Recommended:
                  706016001
                  372860001
                  706016002
User 00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e
     Known positives:
                  732413001
                  742079001
     Recommended:


## MAP@12 Calculation for entire dataset

In [22]:
sparse_customer_article_train, sparse_customer_article_test = random_train_test_split(customer_to_article_interaction, test_percentage=0.2, random_state=42)

In [23]:
sparse_customer_article_train

<1362281x104547 sparse matrix of type '<class 'numpy.float32'>'
	with 21845151 stored elements in COOrdinate format>

In [24]:
sparse_customer_article_test

<1362281x104547 sparse matrix of type '<class 'numpy.float32'>'
	with 5461288 stored elements in COOrdinate format>

### Cross Validation methodolody

In [25]:
# Initialising model with warp loss function
model_without_features = LightFM(loss = "warp")

# Fitting into user to product interaction matrix only / pure collaborative filtering factor
start = time.time()
model_without_features.fit(sparse_customer_article_train,
                          user_features=None, 
                          item_features=None, 
                          sample_weight=None, 
                          epochs=1, 
                          num_threads=4,
                          verbose=False)
end = time.time()
print("time taken = {0:.{1}f} seconds".format(end - start, 2))

time taken = 16.98 seconds


### Writing Precision Calculation

In [26]:
# Precision metric score (ranging from 0 to 1)
'''
k = 12
precision_without_features = []
for precision_k in tqdm.tqdm(range(1,k+1), desc='Calculating Precisions at different k levels'):
    start = time.time()
    precision_without_features.append(precision_at_k(model = model_without_features, 
                                                    test_interactions = sparse_customer_article_test,
                                                    num_threads = 4, 
                                                    k=precision_k,
                                                    check_intersections = True))
    end = time.time()
    print("Time taken for top %s number precision = %s seconds" %(str(precision_k), str(round(end-start,2))))
'''

'\nk = 12\nprecision_without_features = []\nfor precision_k in tqdm.tqdm(range(1,k+1), desc=\'Calculating Precisions at different k levels\'):\n    start = time.time()\n    precision_without_features.append(precision_at_k(model = model_without_features, \n                                                    test_interactions = sparse_customer_article_test,\n                                                    num_threads = 4, \n                                                    k=precision_k,\n                                                    check_intersections = True))\n    end = time.time()\n    print("Time taken for top %s number precision = %s seconds" %(str(precision_k), str(round(end-start,2))))\n'

In [27]:
'''
from numpy import save
save('./precision_without_features.npy', precision_without_features)
'''

"\nfrom numpy import save\nsave('./precision_without_features.npy', precision_without_features)\n"

In [28]:
precision_without_features = np.load('../input/hm-trained-models/lightfm_nofeatures/precision_without_features.npy')

### Calculating Average Precision@12

In [29]:
map_12 = np.sum(precision_without_features, axis=0) / 12

### Calculating Mean Average Precision

In [30]:
print("average precision @ 12 without adding item-feature interaction = {0:.{1}f}".format(map_12.mean(), 3)) 

average precision @ 12 without adding item-feature interaction = 0.005


## Saving Final Model with any feature

In [31]:
with open('model_without_feature.pickle', 'wb') as fle:
    pickle.dump(final_model_without_feature, fle, protocol=pickle.HIGHEST_PROTOCOL)

## Getting Predictions based on submission template

In [32]:
submission_data = pd.read_csv(submission_data_path)
submission_data.shape

(1371980, 2)

In [33]:
submission_data.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [34]:
submission_data.loc[submission_data.customer_id.isin([submission_data.customer_id.unique()[0]])].prediction[0].split(' ')

['0706016001',
 '0706016002',
 '0372860001',
 '0610776002',
 '0759871002',
 '0464297007',
 '0372860002',
 '0610776001',
 '0399223001',
 '0706016003',
 '0720125001',
 '0156231001']

In [35]:
def create_chunk_indices(meta_df, chunk_idx, chunk_size):
    '''
    Function to generate chunks of data for multiprocessing
    '''
    start_idx = chunk_idx * chunk_size
    end_idx = start_idx + chunk_size
    meta_chunk = meta_df[start_idx:end_idx]
    print("start/end "+str(chunk_idx+1)+":" + str(start_idx) + "," + str(end_idx))
    print(len(meta_chunk))
    #chunk_idx in return value is used to sort the processed chunks back into original order,
    return (meta_chunk, chunk_idx)

In [36]:
def predict_sub_chunks(chunk):
    final_submission=[]
    for row in tqdm.tqdm(chunk[0].values):
        try:
            preds = recom_without_feature.get_batched_recommendation(row[0], 12)
            if preds is not None:
                final_submission.append(' '.join(map(str, recom_without_feature.get_batched_recommendation(row[0], 12))))
            else:
                final_submission.append(row[1])
        except Exception as ex:
            print(ex)
    return final_submission

In [66]:
num_cores=4

def predict_submission(submission_data=None):
    #splitting here by measurement id's to get all signals for a measurement into single chunk
    customer_ids = submission_data["customer_id"].unique()
    df_split = np.array_split(customer_ids, num_cores)
    chunk_size = len(df_split[0])
    
    chunk1 = create_chunk_indices(submission_data, 0, chunk_size)
    chunk2 = create_chunk_indices(submission_data, 1, chunk_size)
    chunk3 = create_chunk_indices(submission_data, 2, chunk_size)
    chunk4 = create_chunk_indices(submission_data, 3, chunk_size)
    
    #list of items for multiprocessing, 4 since using 4 cores
    all_chunks = [chunk1, chunk2, chunk3, chunk4]
    
    pool = Pool(num_cores)
    result = pool.map(predict_sub_chunks, all_chunks)
    
    result_combined = list(itertools.chain(result[0], result[1], result[2], result[3]))
    return result_combined

In [67]:
final_predictions = predict_submission(submission_data[:16])

start/end 1:0,4
4
start/end 2:4,8
4
start/end 3:8,12
4
start/end 4:12,16
4


100%|██████████| 4/4 [00:00<00:00, 18.38it/s]


In [68]:
final_predictions

['372860001 706016001 372860002 610776002 706016002 610776001 562245046 720125001 673677002 562245001 579541001 568601006',
 '706016001 706016002 759871002 372860001 610776002 372860002 610776001 590928001 562245001 706016003 464297007 673396002',
 '706016001 372860001 706016002 759871002 464297007 610776002 372860002 720125001 706016003 399256001 562245046 610776001',
 '706016001 706016002 372860001 759871002 372860002 610776002 464297007 673677002 562245046 568601006 399223001 610776001',
 '706016001 759871002 706016002 372860001 372860002 610776002 464297007 720125001 448509014 673677002 610776001 706016003',
 '706016001 706016002 372860001 759871002 610776002 372860002 464297007 706016003 673396002 610776001 562245046 562245001',
 '706016001 706016002 759871002 372860001 610776002 372860002 610776001 706016003 568601006 673677002 608776002 464297007',
 '706016001 706016002 372860001 759871002 464297007 372860002 610776002 399223001 673677002 720125001 673396002 562245046',
 '706016

## Writing Predictions

In [69]:
final_submission_data = submission_data.copy()
final_submission_data['prediction'] = final_predictions

ValueError: Length of values (16) does not match length of index (1371980)

In [61]:
final_submission_data.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,None
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,None
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,None
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,None
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,None


In [ ]:
final_submission_data.to_csv("submission.csv", index=False)